### Building a RAG System with LangChain and FAISS
Introduction to RAG (Retrieval-Augmented Generation)
RAG combines the power of retrieval systems with generative AI models. Instead of relying solely on the model's training data, RAG:

1. Retrieves relevant documents from a knowledge base
2. Uses these documents as context for the LLM
3. Generates responses based on both the retrieved context and the model's knowledge

### FAISS
https://github.com/facebookresearch/faiss

FAISS is a library for efficient similarity search and clustering of dense vectors.

Key advantages:
1. Extremely fast similarity search
2. Memory efficient
3. Supports GPU acceleration
4. Can handle millions of vectors

How it works:
- Indexes vectors for fast nearest neighbor search
- Returns most similar vectors based on distance metrics


In [2]:
## load libraries
import os
from dotenv import load_dotenv
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# LangChain core imports
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import (
    RunnablePassthrough,

)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# LangChain specific imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

# Load environment variables
load_dotenv()

True

### Data Ingestion And Processing

In [3]:
sample_documents = [
    Document(
        page_content="""
        Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
        """,
        metadata={"source": "AI Introduction", "page": 1, "topic": "AI"}
    ),
    Document(
        page_content="""
        Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
        """,
        metadata={"source": "ML Basics", "page": 1, "topic": "ML"}
    ),
    Document(
        page_content="""
        Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
        """,
        metadata={"source": "Deep Learning", "page": 1, "topic": "DL"}
    ),
    Document(
        page_content="""
        Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sentiment analysis, and text summarization.
        """,
        metadata={"source": "NLP Overview", "page": 1, "topic": "NLP"}
    )
]

print(sample_documents)

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='\n        Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.\n        '), Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='\n        Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.\n        '), Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='\n        Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolu

In [4]:
## text splitting
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[" "]
)

## split the documents into chunks
chunks = text_splitter.split_documents(sample_documents)
print(chunks[0])
print(chunks[1])

page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.' metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}
page_content='Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.' metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}


In [5]:
print(f"Created {len(chunks)} chunks from {len(sample_documents)} documents")
print("\nExample chunk:")
print(f"Content: {chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

Created 4 chunks from 4 documents

Example chunk:
Content: Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
Metadata: {'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}


In [6]:
# Initialize hugging fase embeddings with the latest model

embeddings=HuggingFaceEmbeddings(
    model="sentence-transformers/all-MiniLM-L6-v2"
)

## Example: create a embedding for a single text
sample_text="What is machine learning"
sample_embedding=embeddings.embed_query(sample_text)
sample_embedding

[-0.029035672545433044,
 0.0075597031973302364,
 0.04076486453413963,
 0.030535517260432243,
 0.051757119596004486,
 -0.017347561195492744,
 -0.03094184398651123,
 -0.06556293368339539,
 -0.03306081146001816,
 -0.009561285376548767,
 -0.09131385385990143,
 0.0473836213350296,
 0.027739526703953743,
 -0.06366026401519775,
 -0.0650474950671196,
 0.042002685368061066,
 -0.040218982845544815,
 0.02800508588552475,
 -0.028049809858202934,
 -0.05376286432147026,
 -0.005428343079984188,
 0.006084781140089035,
 -0.07086585462093353,
 0.025930067524313927,
 0.010079488158226013,
 0.026293471455574036,
 0.03857466205954552,
 0.02275240607559681,
 -0.01670047454535961,
 0.009418035857379436,
 0.016376059502363205,
 -0.05924325808882713,
 -0.01635243557393551,
 0.045799050480127335,
 -0.061201781034469604,
 0.061170998960733414,
 -0.013366841711103916,
 -0.00031113126897253096,
 0.039353422820568085,
 -0.04183920472860336,
 -0.039276741445064545,
 -0.1010734811425209,
 -0.006085107568651438,
 -0.0

In [7]:
texts=["AI","MAchine learning","Deep Learning","Neural Network"]
batch_embeddings=embeddings.embed_documents(texts)
print(batch_embeddings[0])

[-0.036539338529109955, -0.015164421871304512, 0.016432451084256172, 0.010568785481154919, 0.006010621786117554, -0.01847320795059204, 0.08546525985002518, 0.02096845582127571, 0.027815328910946846, 0.012431614100933075, -0.02937653474509716, -0.031135281547904015, 0.03491254895925522, -0.018150819465517998, -0.06498484313488007, 0.05168251320719719, -0.01960616372525692, -0.015734253451228142, -0.13371676206588745, -0.09645994752645493, -0.025471746921539307, -0.0014895017957314849, -0.006349325645714998, -0.025820719078183174, -0.02737177163362503, 0.12268999218940735, -0.007792511489242315, -0.038522735238075256, 0.014383546076714993, -0.0921841710805893, 0.008695710450410843, 0.0026133558712899685, 0.0910346582531929, -0.03031359426677227, -0.09604638814926147, 0.022289054468274117, -0.09024312347173691, -0.032947417348623276, 0.07158327847719193, -0.00889308750629425, -0.025708893314003944, -0.07913961261510849, 0.01453041285276413, -0.07420427352190018, 0.08045008033514023, 0.078

In [8]:
print(batch_embeddings[1])

[-0.02439202554523945, 0.0032444121316075325, 0.054267678409814835, -0.006672615651041269, 0.003935693763196468, -0.007957442663609982, 0.025025134906172752, -0.03203273192048073, -0.0545106902718544, -0.044702064245939255, -0.013759499415755272, 0.0160613302141428, 0.040364667773246765, -0.020260930061340332, -0.0609746128320694, 0.020655613392591476, 0.010556417517364025, -0.016264749690890312, -0.10490721464157104, -0.1106831282377243, -0.021544678136706352, -0.013036052696406841, -0.08688350021839142, 0.027151934802532196, 0.026144007220864296, 0.039646558463573456, 0.06494351476430893, 0.06547266244888306, 0.017963258549571037, -0.10655659437179565, 0.009878230281174183, -0.034961994737386703, 0.03040345571935177, 0.014532884582877159, -0.11560288816690445, 0.012346232309937477, -0.06430946290493011, 0.04394593834877014, 0.019033189862966537, 0.030984794721007347, -0.015413874760270119, -0.08163449913263321, 0.012414462864398956, 0.012423690408468246, 0.06950365751981735, 0.077825

In [9]:
### Compare Embedding using cosine similarity

def compare_embeddings(text1:str,text2:str):
    """Compare semantic simialrity of 2 texts usign embeddings"""

    emb1=np.array(embeddings.embed_query(text1))
    emb2=np.array(embeddings.embed_query(text2))

    ## Calculate the simialrity score

    similarity=np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similarity

In [10]:
# Test semantic similarity
print("\nSemantic Similarity Examples:")
print(f"'AI' vs 'Artificial Intelligence': {compare_embeddings('AI', 'Artificial Intelligence'):.3f}")


Semantic Similarity Examples:
'AI' vs 'Artificial Intelligence': 0.791


In [11]:
print(f"'AI' vs 'Pizza': {compare_embeddings('AI', 'Pizza'):.3f}")

'AI' vs 'Pizza': 0.257


In [12]:
print(f"'Machine Learning' vs 'ML': {compare_embeddings('Machine Learning', 'ML'):.3f}")

'Machine Learning' vs 'ML': 0.373


### Create FAISS Vector Store

In [16]:
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)
print(f"Vector store created with {vectorstore.index.ntotal} vectors")

Vector store created with 4 vectors


In [17]:
vectorstore

In [18]:
## Save vector tore for later use
vectorstore.save_local("faiss_index")
print("Vector store saved to 'faiss_index' directory")

Vector store saved to 'faiss_index' directory


In [21]:
# load vector store
loaded_vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)
print(f"Loaded vector store contains {loaded_vectorstore.index.ntotal} vectors")

Loaded vector store contains 4 vectors


In [22]:
## Similarity Search
query="What is deep learning"

results=vectorstore.similarity_search(query,k=3)
print(results)

[Document(id='b1ee786f-671d-46ab-bfc6-725668e2dbc4', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recognition.'), Document(id='94356327-e0d8-4475-b53b-b9442c6973ac', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'), Document(id='eb54fb3b-d302-4d32-876d-ed061a8a75e2', metadata={'source': 'NLP Overview', 'page': 1, 'topic': 'NLP'}, page_content='Natural Language Processing (NLP) is a branch of AI that helps computers understand human lang

In [23]:
print(f"Query: {query}\n")
print("Top 3 similar chunks:")
for i, doc in enumerate(results):
    print(f"\n{i+1}. Source: {doc.metadata['source']}")
    print(f"   Content: {doc.page_content[:200]}...")

Query: What is deep learning

Top 3 similar chunks:

1. Source: Deep Learning
   Content: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning ...

2. Source: ML Basics
   Content: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised...

3. Source: NLP Overview
   Content: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
      ...


In [24]:
### Similarity Search with score
results_with_scores=vectorstore.similarity_search_with_score(query,k=3)

print("\n\nSimilarity search with scores:")
for doc, score in results_with_scores:
    print(f"\nScore: {score:.3f}")
    print(f"Source: {doc.metadata['source']}")
    print(f"Content preview: {doc.page_content[:100]}...")



Similarity search with scores:

Score: 0.343
Source: Deep Learning
Content preview: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses m...

Score: 1.090
Source: ML Basics
Content preview: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being...

Score: 1.154
Source: NLP Overview
Content preview: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
...


In [25]:
chunks

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.'),
 Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'),
 Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recogn

In [26]:
### Search with metadata filtering
filter_dict={"topic":"ML"}
filtered_results=vectorstore.similarity_search(
    query,
    k=3,
    filter=filter_dict
)
print(filtered_results)

[Document(id='94356327-e0d8-4475-b53b-b9442c6973ac', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.')]


In [27]:
len(filtered_results)

1

### Build RAG Chain With LCEL

In [30]:
## LLM GROQ LLM
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model(model="groq:openai/gpt-oss-20b")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x30ce2e990>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x30ce2f390>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [31]:
llm.invoke("Hi")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'reasoning_content': 'User says "Hi". Probably wants a friendly response. Just greet back.'}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 72, 'total_tokens': 106, 'completion_time': 0.034162884, 'prompt_time': 0.005703147, 'queue_time': 0.053643452, 'total_time': 0.039866031, 'completion_tokens_details': {'reasoning_tokens': 16}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--cdf4e1fb-4461-4ada-b383-39bdfe3467a1-0', usage_metadata={'input_tokens': 72, 'output_tokens': 34, 'total_tokens': 106})

In [32]:
# 1. Simple RAG Chain with LCEL
simple_prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context:
Context: {context}

Question: {question}

Answer:""")

In [33]:
## Basic retriever
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [34]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x17d55f610>, search_kwargs={'k': 3})

In [35]:
from typing import List
# Format documents for the prompt
def format_docs(docs: List[Document]) -> str:
    """Format documents for insertion into prompt"""
    formatted = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1} (Source: {source}):\n{doc.page_content}")
    return "\n\n".join(formatted)

In [36]:
simple_rag_chain=(
    {"context":retriever | format_docs,"question":RunnablePassthrough() }
    | simple_prompt
    | llm
    |StrOutputParser()

)

In [37]:
simple_rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x17d55f610>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\nContext: {context}\n\nQuestion: {question}\n\nAnswer:'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x30ce2e990>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x30ce2f390>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [39]:
### Conversational RAg Chain

conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer questions."),
    ("placeholder", "{chat_history}"),
    ("human", "Context: {context}\n\nQuestion: {input}"),
])

In [40]:
def create_conversational_rag():
    """Create a conversational RAG chain with memory"""
    return (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["input"]))
        )
        | conversational_prompt
        | llm
        | StrOutputParser()
    )

conversational_rag = create_conversational_rag()

In [41]:
conversational_rag

RunnableAssign(mapper={
  context: RunnableLambda(lambda x: format_docs(retriever.invoke(x['input'])))
})
| ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag=

In [42]:
### streaming RAG chain
streaming_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | simple_prompt
    | llm
)

print("Modern RAG chains created successfully!")
print("Available chains:")
print("- simple_rag_chain: Basic Q&A")
print("- conversational_rag: Maintains conversation history")
print("- streaming_rag_chain: Supports token streaming")

Modern RAG chains created successfully!
Available chains:
- simple_rag_chain: Basic Q&A
- conversational_rag: Maintains conversation history
- streaming_rag_chain: Supports token streaming


In [43]:
# Test function for different chain types
def test_rag_chains(question: str):
    """Test all RAG chain variants"""
    print(f"Question: {question}")
    print("=" * 80)

    # 1. Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke(question)
    print(f"Answer: {answer}")

    print("\n2. Streaming RAG:")
    print("Answer: ", end="", flush=True)
    for chunk in streaming_rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print()

In [45]:
test_rag_chains("What is the difference between AI and machine learning")

Question: What is the difference between AI and machine learning

1. Simple RAG Chain:
Answer: AI is the broader field that seeks to simulate human intelligence in machines—systems that can think and act like humans.  
Machine learning is a subset of AI; it focuses specifically on algorithms that learn patterns from data instead of being explicitly programmed.

2. Streaming RAG:
Answer: AI is the broader field that seeks to simulate human intelligence in machines—systems that can think and act like humans.  
Machine learning (ML) is a specific subset of AI that focuses on enabling systems to learn patterns from data without being explicitly programmed. In other words, all ML is AI, but not all AI is ML.


In [46]:
# Test with multiple questions
test_questions = [
    "What is the difference between AI and Machine Learning?",
    "Explain deep learning in simple terms",
    "How does NLP work?"
]

for question in test_questions:
    print("\n" + "=" * 80 + "\n")
    test_rag_chains(question)



Question: What is the difference between AI and Machine Learning?

1. Simple RAG Chain:
Answer: Artificial Intelligence (AI) is the broad field that aims to simulate human intelligence in machines—enabling systems to think and act like humans. Machine Learning (ML) is a specific subset of AI that focuses on algorithms that learn patterns from data instead of being explicitly programmed for each task. In short, all machine learning is AI, but not all AI is machine learning.

2. Streaming RAG:
Answer: Artificial Intelligence (AI) is the broad concept of machines that simulate human intelligence—thinking and acting like humans. Machine Learning (ML) is a specific subset of AI that gives systems the ability to learn patterns from data instead of being explicitly programmed. In short, AI is the overall field, while ML is one of its techniques.


Question: Explain deep learning in simple terms

1. Simple RAG Chain:
Answer: Deep learning is a type of machine learning that uses artificial ne

In [47]:
## Conversational example
print("\n3. Conversational RAG Example:")
chat_history = []

# First question
q1 = "What is machine learning?"
a1 = conversational_rag.invoke({
    "input": q1,
    "chat_history": chat_history
})

print(f"Q1: {q1}")
print(f"A1: {a1}")


3. Conversational RAG Example:
Q1: What is machine learning?
A1: **Machine learning** is a subset of artificial intelligence that enables systems to learn from data rather than being explicitly programmed. In practice, ML algorithms discover patterns in data and use those patterns to make predictions or decisions. The main categories of machine learning are **supervised learning**, **unsupervised learning**, and **reinforcement learning**.


In [48]:
# Update history
chat_history.extend([
    HumanMessage(content=q1),
    AIMessage(content=a1)
])

In [49]:
# Follow-up question
q2 = "How is it different from traditional programming?"
a2 = conversational_rag.invoke({
    "input": q2,
    "chat_history": chat_history
})
print(f"\nQ2: {q2}")
print(f"A2: {a2}")


Q2: How is it different from traditional programming?
A2: **Key differences between machine learning and traditional (rule‑based) programming**

| Aspect | Traditional Programming | Machine Learning |
|--------|------------------------|------------------|
| **How behavior is defined** | Explicit, hand‑written rules and logic written by a programmer. | Statistical patterns learned automatically from data. |
| **Development process** | The programmer writes code that covers all possible cases (or uses a deterministic algorithm). | The programmer builds a model and supplies labeled or unlabeled data; the model “learns” the mapping from input to output. |
| **Adaptability** | Hard‑coded logic; changes require editing the source code and redeploying. | The model can improve or adapt by being retrained on new data without changing the underlying code. |
| **Error handling** | Errors are usually deterministic; a bug is traced to a specific line of code. | Errors arise from distributional dif